# h2o in python

In [ ]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

In [94]:
#將h2o初始化
h2o.init(nthreads =6)  

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 0 mins
H2O cluster timezone:,Asia/Taipei
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,17 days
H2O cluster name:,H2O_started_from_R_user_orx046
H2O cluster total nodes:,1
H2O cluster free memory:,234.4 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [96]:
##導入h2o內建隨機森林套件
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [ ]:
##輸入資料URL
loan_csv ="https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
##可以利用此函數直接使用URL導入資料(與R函數不同)
data = h2o.import_file(path=loan_csv)

In [ ]:
##看資料型態
data.head()

In [ ]:
##data放入自己取名之檔案名
##.split_frame代表要分割資料
train,test = data.split_frame(ratios=[0.7], seed=1)  

In [ ]:
print(train.nrow)
print(test.nrow)

In [ ]:
##刪除缺失值
train=train.na_omit()
test=test.na_omit()
print(train.nrow)
print(test.nrow)

In [ ]:
import pandas as pd

train['bad_loan']=train['bad_loan'].asfactor()
test['bad_loan']=test['bad_loan'].asfactor() 
print(type(train))

In [ ]:
##設置x、y的變數名
y = 'bad_loan'
x = list(data.columns)
x.remove(y)
x.remove('int_rate')
x.remove('addr_state')
x.remove('home_ownership')
x

In [ ]:
##建立隨機森林模型
model_h2o = H2ORandomForestEstimator(ntrees=500,seed=1)

In [ ]:
##使用訓練集訓練模型
model_h2o.train(x=x,y=y,training_frame=train)
model_h2o.train

In [62]:
##看模型表現
performance = model_h2o.model_performance(test_data=test)
performance


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.14151577467745965
RMSE: 0.376185824663104
MAE: 0.28207336380723136
RMSLE: 0.2645978753246043
Mean Residual Deviance: 0.14151577467745965


In [53]:
predict_rf_h2o = model_h2o.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


predict
0.417819
0.0845455
0.285437
0.076861
0.206525


In [54]:
(predict_rf_h2o['predict']==test['bad_loan']).as_data_frame(use_pandas=True).mean()

predict    0.0
dtype: float64

In [99]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-99-06c455af4589> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
